In [ ]:
%%cmd
pip install --upgrade pip
pip install -r requirements.txt 

In [ ]:
%jupyter nbextension enable --py widgetsnbextension

In [11]:
import sys
display(sys.executable)
display(sys.prefix)

'd:\\Documents\\work\\vscode_workspace\\gpt-persona\\.venv_persona\\Scripts\\python.exe'

'd:\\Documents\\work\\vscode_workspace\\gpt-persona\\.venv_persona'

In [12]:
import os
import openai

openai.api_key = os.getenv("OPENAI_API_KEY")

In [13]:


class Scene:
    def __init__(self, name: str, common_context: str, agent_contexts: dict = {}):
        self.name = name 
        self.common_context = common_context
        self.agent_contexts = agent_contexts

    def get_scene_name(self):
        return self.name

    def add_agent_context(self, agent_id, context):
        self.agent_contexts[agent_id] = context 

    def get_common_context(self):
        return self.common_context

    def get_agent_context(self, agent_id):
        return self.agent_contexts.get(agent_id, None)



class Session:
    def __init__(self, session_id:str, history:list = []):
        self.session_id = session_id
        self.history = history
    

    def insert_talk(self, talk:dict):
        self.history.append(talk)
    
    def get_history(self):
        return self.history

    def get_formatted_history(self):
        text = ""
        for talk in self.history:
            text += f"{talk['agent_name']}: {talk['text']}\n"
        return text 

   

class Agent:
    def __init__(self, 
                 agent_id: str,
                 agent_name: str,
                 description: str,
                 long_term_memory: dict={}, 
                 personality: dict={},
                 ):
        self.agent_id = agent_id
        self.agent_name = agent_name
        self.description = description
        self.long_term_memory = long_term_memory
        self.personality = personality
        
    
    def speak(self, text, session: Session):
        session.insert_talk({'agent_name': self.agent_name, 
                             'text': text
                             })

    def build_prompt_with_context(self, scene: Scene, session: Session):
        system_prompt = {
            "role": "system",
            "content": f"""You are a person with name {self.agent_name}. """
        }
        user_prompt = {
            "role": "user",
            "content": f"""
                Your name is {self.agent_name}. 
                {self.description}
                You are in the scene {scene.get_scene_name()}. 
                {scene.get_common_context()}
                {scene.get_agent_context(self.agent_id)}
                Below delimted by three equal signs === is the history of the conversion so far. 
                ===
                {session.get_formatted_history()}
                ===
                Please generate output with 3 fields in json format like. 
                The 1st field is: do you want to finish the conversation? Please answer "Yes" or "No".
                The 2nd field is: if you want to finish the conversation, is an agreement reached or not? Please answer with "Yes" or "No". If you don't want finish the conversatioin, please answer blank value ''.
                The 3rd field is: what is your reply to conversation?
                An example output is provided below: 
                {{
                    "want_to_finish": "No",
                    "agreement": "No",
                    "reply": your reply goes here 
                }}
            """
        }
        return [system_prompt,
                user_prompt]


In [3]:
#openai.Model.list()

In [35]:
agent_sales = Agent(agent_id="sales",
                    agent_name="Oliver",
                    description="You are a sales man at Bing Lee, which is an Australian chain superstores",
                    
                    )
agent_buyer = Agent(agent_id="buyer",
                    agent_name="Charlotte",
                    description="You are a new university student. You want to buy a fridge for your dorm."
                    )
scene_binglee = Scene(name="binglee",
                      common_context="It is the fridge section in Bing Lee Rhodes store. Oliver is the saleman, and Charlotte wants to buy a fridge for her dorm in unversity. Charlotte is looking at LG 420L BOTTOM MOUNT FRIDGE price aud 988.",
                      )
scene_binglee.add_agent_context(agent_id=agent_sales.agent_id, context="You have the authority to give up to 10% discount, and upto AUd 30 discount on delivery fee. Deliver fee is Aud 40  0-10km, aud 60 10-20km. No deliver for more than 20km.")
scene_binglee.add_agent_context(agent_id=agent_buyer.agent_id, context="You have budget aud 1000 to buy a fridge. You plan to bargin for a good price, saving money as much as possible.")
session_binglee = Session(session_id="bing_lee",
                          history=[{"agent_name": "Oliver",
                                    "text": "Hi, my name is Oliver. How can I help you?"
                                    },
                                   {"agent_name": "Charlotte",
                                    "text": "Hi, I am charlootte. I am looking for a fridge for my dorm. This LG 420L Bottom Mount Fridge looks good. Can you give me a discount?"
                                    }
                                   ]
                          )


records = []
records.extend(session_binglee.get_history())

In [36]:
import pprint
def carry_out_one_turn(scene, session, agent):
    messages = agent.build_prompt_with_context(
        scene=scene,
        session=session
    )

    display(f"message length: {len(messages[1]['content'])}")

    response = openai.ChatCompletion.create(
                model='gpt-3.5-turbo',
                messages=messages,
                max_tokens=1000,
                temperature=0.2
            )
    reply = response["choices"][0]["message"]['content']
    reply_json = json.loads(reply)
    pprint.pprint(reply_json)
    records.append({"agent_name": agent.agent_name,
                    "record": reply_json})

    if "reply" in reply_json:
        session.insert_talk({"agent_name":agent.agent_name,
                            "text": reply_json.get("reply")
                            })
    return reply_json

In [58]:
# Salesman talk
import json
import pprint
scene = scene_binglee
session = session_binglee

agent = agent_sales

carry_out_one_turn(scene=scene, session=session, agent=agent)


{'agreement': 'No',
 'reply': "I'm sorry, Charlotte. Based on the discussions with my manager, we "
          'are unable to offer a higher discount or explore other options to '
          'bring the price within your budget. I understand your need for a '
          'fridge, and I apologize for any inconvenience caused. If you have '
          "any other questions or if there's anything else I can assist you "
          'with, please let me know.',
 'want_to_finish': 'Yes'}


In [60]:
# buyer talk
import json
import pprint
scene = scene_binglee
session = session_binglee

agent = agent_buyer


carry_out_one_turn(scene=scene, session=session, agent=agent)

{'agreement': 'No',
 'reply': 'Oliver, I understand that you may have limitations on the discounts '
          'you can offer. However, since my budget is AUD 1000 and the current '
          'price of the fridge is still higher than that, I was wondering if '
          "there's any way we can further negotiate the price or explore other "
          'options to bring it within my budget. I really need a fridge for my '
          'dorm, and I would love to purchase it from Bing Lee if we can find '
          'a mutually agreeable solution.',
 'want_to_finish': 'No'}


4732

In [37]:
import ipywidgets as widgets
from IPython.display import display
import json
import pprint
scene = scene_binglee
session = session_binglee


last_talk_agent = agent_sales


# 1. Textarea widget to display chat history
chat_history = widgets.Textarea(
    value="",
    placeholder="Chat history will appear here...",
    description="Chat:",
    disabled=True,
    layout=widgets.Layout(width="500px", height="300px")
)

for talk in session.get_history():
    chat_history.value += f"{talk['agent_name']}: {talk['text']}\n\n" 


# 2. Button widget to send message
send_button = widgets.Button(description="Send")

# 3. Function to handle button click and update chat
def on_send_button_click(button):
    global last_talk_agent
    
    if last_talk_agent == agent_sales:
        current_talk_agent = agent_buyer
    else:
        current_talk_agent = agent_sales

    reply_json = carry_out_one_turn(scene=scene, session=session, agent=current_talk_agent)
    print(reply_json)
    reply = reply_json.get("reply", None)
    want_to_finish = reply_json.get("want_to_finish", None)
    agreement = reply_json.get("agreement", None)
    if reply is None:
        print(f"No reply: f{reply_json}")
    else:
        # Append OpenAI's response to chat history
        chat_history.value += f"{current_talk_agent.agent_name}: {reply_json['reply']}\n\n"
    if want_to_finish is not None and 'yes' in want_to_finish.lower():
        chat_history.value += f"SYSTEM: {current_talk_agent.agent_name} wants to finish conversation.\n\n"
        if agreement is not None:
            chat_history.value += f"SYSTEM: agreement is {'' if 'yes' in agreement.lower() else 'not'} reached."
    last_talk_agent = current_talk_agent

    


# Set button click event handler
send_button.on_click(on_send_button_click)

# Display the widgets
display(chat_history, send_button)

Textarea(value='Oliver: Hi, my name is Oliver. How can I help you?\n\nCharlotte: Hi, I am charlootte. I am loo…

Button(description='Send', style=ButtonStyle())

'message length: 1576'

{'agreement': '',
 'reply': "Hi Oliver, nice to meet you. I'm Charlotte. I'm interested in this "
          'LG 420L Bottom Mount Fridge. Is there any possibility of getting a '
          'discount on the price?',
 'want_to_finish': ''}
{'want_to_finish': '', 'agreement': '', 'reply': "Hi Oliver, nice to meet you. I'm Charlotte. I'm interested in this LG 420L Bottom Mount Fridge. Is there any possibility of getting a discount on the price?"}


'message length: 1799'

...

{'agreement': 'No',
 'reply': 'Thank you, Oliver. I appreciate your efforts in trying to '
          "accommodate my budget. If there's any possibility of further "
          'negotiation on the price, I would be grateful. Is there anything '
          'else we can explore to find a solution that works for both of us?',
 'want_to_finish': 'No'}
{'want_to_finish': 'No', 'agreement': 'No', 'reply': "Thank you, Oliver. I appreciate your efforts in trying to accommodate my budget. If there's any possibility of further negotiation on the price, I would be grateful. Is there anything else we can explore to find a solution that works for both of us?"}


'message length: 3863'

{'agreement': 'No',
 'reply': 'I apologize, Charlotte, but after discussing with my manager, we '
          'are unable to offer any further discount on the LG 420L Bottom '
          'Mount Fridge. However, I can still waive the delivery fee of AUD 40 '
          'for you. This would bring the total price to AUD 948. Would you '
          'like to proceed with this offer?',
 'want_to_finish': 'No'}
{'want_to_finish': 'No', 'agreement': 'No', 'reply': 'I apologize, Charlotte, but after discussing with my manager, we are unable to offer any further discount on the LG 420L Bottom Mount Fridge. However, I can still waive the delivery fee of AUD 40 for you. This would bring the total price to AUD 948. Would you like to proceed with this offer?'}


'message length: 4109'

{'agreement': 'No',
 'reply': 'Thank you for your efforts, Oliver. I appreciate the waived '
          'delivery fee, but I was really hoping to find a fridge within my '
          'budget of AUD 1000. I will continue to look for other options. '
          'Thank you for your assistance.',
 'want_to_finish': 'Yes'}
{'want_to_finish': 'Yes', 'agreement': 'No', 'reply': 'Thank you for your efforts, Oliver. I appreciate the waived delivery fee, but I was really hoping to find a fridge within my budget of AUD 1000. I will continue to look for other options. Thank you for your assistance.'}


In [ ]:
with open("./data/binglee_conversation_1.json", "w") as f:
    json.dump(records, f, indent=4)

In [31]:
records

[{'agent_name': 'Oliver',
  'text': 'Hi, my name is Oliver. How can I help you?'},
 {'agent_name': 'Charlotte',
  'text': 'Hi, I like this LG 420L Bottom Mount Fridge. Can you give me a discount?'},
 {'agent_name': 'Charlotte',
  'record': {'want_to_finish': 'No',
   'agreement': 'No',
   'reply': "Yes, I really like this fridge and I'm interested in buying it. However, I have a budget of AUD 1000. Is there any way you can offer me a discount to meet my budget?"}},
 {'agent_name': 'Charlotte',
  'record': {'want_to_finish': 'No',
   'agreement': 'No',
   'reply': 'Oliver: I understand that you have a budget of AUD 1000. Let me see what I can do for you. Please give me a moment to check if there are any ongoing promotions or discounts available for this fridge.'}},
 {'agent_name': 'Oliver',
  'record': {'want_to_finish': 'No',
   'agreement': 'No',
   'reply': 'Sure, let me check if there are any discounts or promotions available for the LG 420L Bottom Mount Fridge.'}},
 {'agent_name': 